In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True)
cdf.printSchema()
cdf.show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



### 복잡한 연산도 해보자
- 일반 select나 groupby 절만 이용해서는 불가능한 경우 사용
- query내에 다른 query포함

In [8]:
schema = StructType([
        StructField('class_cd',StringType()),
        StructField('school',StringType()),
        StructField('class_std_cnt',IntegerType()),    
        StructField('loc',StringType()),
        StructField('school_type',StringType()),
        StructField('teaching_type',StringType())   
])
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True, schema=schema)
cdf.printSchema()
cdf.createOrReplaceTempView('class')

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



#### case 1. 소속된 반의 개수가 2개 이상인 학교들 중 학생 숫자가 가장 적은 학교를 구해보자

In [36]:
#case 1. 소속된 반의 개수가 2개 이상인 학교들 중 학생 숫자가 가장 적은 학교를 구해보자
# 학생숫자가 가장 작은 학교 구하고
spark.sql('SELECT MIN(class_std_cnt) FROM class').show()

spark.sql('SELECT school, SUM(class_std_cnt) FROM class GROUP BY school').show()
# 이둘을 합쳐야함, 추가로 널도 빼줌
spark.sql('''
    SELECT MIN(std_cnt) FROM (SELECT school, SUM(class_std_cnt) AS std_cnt FROM class GROUP BY school HAVING school IS NOT NULL)
''').show()
# 만약 학교이름이 3중첩된 학교만 결과로 보고싶다?  서브 쿼리 전단계까지(3중첩된 학교리스트와 합).
spark.sql('''
    SELECT school, SUM(class_std_cnt) AS cnt FROM class 
    GROUP BY school HAVING school IS NOT NULL AND COUNT(school) >= 3
''').show()

# 서브쿼리 까지 해서 이중 최솟값 구해보자 그리고 그 한개의 학교 이름도 알고싶다면?

# 1단계 위 결과에서 최솟값을 구한 것
spark.sql('''
    SELECT MIN(cnt) FROM (SELECT school, SUM(class_std_cnt) AS cnt FROM class 
    GROUP BY school HAVING school IS NOT NULL AND COUNT(school) >= 3)
''').show(10)
# 46으로 나온 결과를 다시 위 테이블에서 서치 후 이름가져오기
spark.sql('''
    select school
    from (SELECT school, SUM(class_std_cnt) AS cnt FROM class 
    GROUP BY school HAVING school IS NOT NULL AND COUNT(school) >= 3)
    where cnt == (SELECT MIN(cnt) FROM (SELECT school, SUM(class_std_cnt) AS cnt FROM class 
    GROUP BY school HAVING school IS NOT NULL AND COUNT(school) >= 3))
''').show(10)


+------------------+
|min(class_std_cnt)|
+------------------+
|                10|
+------------------+

+------+------------------+
|school|sum(class_std_cnt)|
+------+------------------+
| VHDHF|                51|
| LAYPA|                57|
| GOOBU|               158|
| UUUQX|                84|
|  null|                28|
| CIMBB|                74|
| UKPGS|               128|
| UAGPU|                87|
| CCAAW|               109|
| FBUMG|                46|
| ZOWMK|               117|
| ZMNYA|                69|
| QOQTS|               149|
| CUQAM|               107|
| OJOBU|                81|
| GOKXL|                64|
| KFZMY|                52|
| ANKYI|                41|
| GJJHK|               118|
| KZKKE|               111|
+------+------------------+
only showing top 20 rows

+------------+
|min(std_cnt)|
+------------+
|          41|
+------------+

+------+---+
|school|cnt|
+------+---+
| VHDHF| 51|
| LAYPA| 57|
| GOOBU|158|
| UUUQX| 84|
| CIMBB| 74|
| UKPGS|128|
| U

#### case2.학생수가 null이 아닌 데이터 중에서 아래 조건에 맞는 학교의 이름과 학생수를 추출하시오
- 반별 학생수의 평균보다 반 학생수가 많은 데이터를 추출하시오

In [58]:
# 내가 연습한것.
# spark.sql('''
#     SELECT class_cd, class_std_cnt FROM class WHERE class_std_cnt IS NOT NULL
# ''').show(5)
# spark.sql('''
#     SELECT class_cd, SUM(class_std_cnt)  
#     FROM(SELECT class_cd, class_std_cnt FROM class WHERE class_std_cnt IS NOT NULL)
#     GROUP BY class_cd
# ''').show(5)

# spark.sql('''
#     SELECT AVG(class_std_cnt) 
#     FROM(SELECT class_cd, SUM(class_std_cnt) 
#     FROM(SELECT class_cd, class_std_cnt FROM class WHERE class_std_cnt IS NOT NULL)
#     GROUP BY class_cd)
# ''').show(5)

In [67]:
spark.sql('''
    SELECT AVG(class_std_cnt) FROM class
''').show(5)

+------------------+
|avg(class_std_cnt)|
+------------------+
|21.828282828282827|
+------------------+



In [59]:
spark.sql('''
    SELECT (*)
    FROM class
    WHERE (class_std_cnt) > (SELECT AVG(class_std_cnt) FROM class) AND (class_std_cnt IS NOT NULL)
''').show(5)

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     BFY| CUQAM|           27|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     X6Z| CUQAM|           24|   Urban|     Public| Experimental|
|     2AP| DNQDD|           27|Suburban|     Public|     Standard|
+--------+------+-------------+--------+-----------+-------------+
only showing top 5 rows



### case2. 소속된 반의 개수가 2개 이상인 학교들 중 학생 숫자가 가장 적은 학교보다
# 학생이 많은 반의 데이터를 구하시오



In [4]:
# where절 subquery 
spark.sql('''
    SELECT school FROM class
    WHERE 
''').show(5)


#### case 3. 소속된 반의 개수가 3개 미만인 학교들 중 학생 숫자가 가장 많은 학교보다, <br> 학생 숫자가 더 적으면서 소속된 반의 개수가 3개 이상인 학교를 구하시오

In [5]:
# 소속된 반의 개수가 3개 미만인 학교들 중 학생 숫자가 가장 많은 학교보다
# 학생 숫자가 더 적으면서 소속된 반의 개수가 3개 이상인 학교를 구하시오



#### sql

In [6]:
# having절 subquery 
# 소속된 반의 개수가 3개 미만인 학교들 중 학생 숫자가 가장 많은 학교보다
# 학생 숫자가 더 적으면서 소속된 반의 개수가 3개 이상인 학교를 구하시오



#### case 4. 도시 지역의 공립학교에서 일반 교육을 진행하는 반의 평균 인원수보다, <br>더 많은 학생이 소속된 교외 지역의 사립학교에서 특수 교육을 진행하는 반을 구하시오

- 도시 지역의 공립학교에서 일반 교육을 진행하는 반의 평균 인원수는 올림처리 합니다.


In [7]:
# 도시 지역의 공립학교에서 일반 교육을 진행하는 반의 평균 인원수보다 
# 더 많은 학생이 소속된 교외 지역의 사립학교에서 특수 교육을 진행하는 반을 구하시오

# 도시 지역의 공립학교에서 일반 교육을 진행하는 반의 평균 인원수는 올림처리 합니다.



#### case 4 sql

In [8]:
# 도시 지역의 공립학교에서 일반 교육을 진행하는 반의 평균 인원수보다 
# 더 많은 학생이 소속된 교외 지역의 사립학교에서 특수 교육을 진행하는 반을 구하시오

# 도시 지역의 공립학교에서 일반 교육을 진행하는 반의 평균 인원수는 올림처리 합니다.

